# **Event set creation for Risk Analysis**
This notebook explains how to create the input files for a risk analysis in the correct format and place them in the correct folder structure. For an explanation on how to derive the (compound) events and their (joint) probabilities, the reader is referred to the FloodAdapt Setup Guide [Event Set](../3_setup_guide/risk_analysis.qmd).


## Import Packages

In [ ]:
import os
import tomli_w
import tomli
from datetime import datetime
from pathlib import Path
from tempfile import gettempdir

from flood_adapt.config.config import Settings
from flood_adapt.config.sfincs import RiverModel
from flood_adapt.dbs_classes.interface.database import IDatabase
from flood_adapt.objects.events.event_set import (
    EventSet,
    SubEventModel,
)
from flood_adapt.objects.events.historical import HistoricalEvent
from flood_adapt.objects.events.synthetic import (
    SyntheticEvent,
)
from flood_adapt.objects.forcing import unit_system as us
from flood_adapt.objects.forcing.discharge import DischargeConstant
from flood_adapt.objects.forcing.forcing import (
    ForcingType,
)
from flood_adapt.objects.forcing.rainfall import RainfallSynthetic
from flood_adapt.objects.forcing.time_frame import (
    TimeFrame,
)
from flood_adapt.objects.forcing.timeseries import (
    ShapeType,
    TimeseriesFactory,
)
from flood_adapt.objects.forcing.waterlevels import (
    SurgeModel,
    TideModel,
    WaterlevelSynthetic,
)
from flood_adapt.objects.forcing.wind import WindConstant

# Configure database
Settings(
    DATABASE_ROOT=Path("../_data/examples").resolve(),
    DATABASE_NAME="charleston_test"
)

## **Step 1**: Set name, frequencies and parameters to build subevents
Set the name of the event set and the parameters for the input events and their frequencies. In this example, we are using three synthetic events with different storm surge heights and peak rainfall intensities. Alternatively, historic gauged and ungauged events or  hurricanes are possible as well, see [Events](events.ipynb) for examples on how to set up different subevents.

#TODO update link to event examples

In [ ]:
name = "event_set"

# Peak surge values in the subevents. Surge is timed to always peak at 24.8 hours after the event start, coinciding with high tide at a tidal phase of 0 and lasting for 10 hours.
surge_values = [2.5, 1, 3] # in meters

# Peak rainfall intensities in the three subevents. Rainfall is applied as a 6 hour "block" shape preceding the peak of the storm surge.
rainfall_values = [10,15,5] # in mm/hr

# Occurrence frequencies of the three events
freq = [0.1,2,0.05]


[SyntheticEvent(name='subevent_1', description='', time=TimeFrame(start_time=datetime.datetime(2020, 1, 1, 0, 0), end_time=datetime.datetime(2020, 1, 3, 0, 0)), template=<Template.Synthetic: 'Synthetic'>, mode=<Mode.single_event: 'single_event'>, forcings={<ForcingType.WIND: 'WIND'>: [WindConstant(type=<ForcingType.WIND: 'WIND'>, source=<ForcingSource.CONSTANT: 'CONSTANT'>, speed=UnitfulVelocity(value=10.0, units=UnitTypesVelocity.mps), direction=UnitfulDirection(value=90.0, units=UnitTypesDirection.degrees))], <ForcingType.RAINFALL: 'RAINFALL'>: [RainfallSynthetic(type=<ForcingType.RAINFALL: 'RAINFALL'>, source=<ForcingSource.SYNTHETIC: 'SYNTHETIC'>, timeseries=BlockTimeseries(shape_type=<ShapeType.block: 'block'>, duration=UnitfulTime(value=6.0, units=UnitTypesTime.hours), peak_time=UnitfulTime(value=21.0, units=UnitTypesTime.hours), peak_value=UnitfulIntensity(value=10.0, units=UnitTypesIntensity.mm_hr), cumulative=None, fill_value=0.0))], <ForcingType.DISCHARGE: 'DISCHARGE'>: [Disc

## **Step 2**: Create subevents
Create the subevents from the parameters defined in Step 1.

In [ ]:

events = []
number = 0
for surge,rainfall in zip(surge_values,rainfall_values):
    number += 1
    events.append(
        SyntheticEvent(
            name = f"subevent_{number}",
            time=TimeFrame(start_time=datetime(2020,1,1,0,0,0),end_time=datetime(2020,1,3,0,0,0)),
            forcings={
                ForcingType.WIND: [         # constant wind speed 10 m/, direction 90 degrees (from the East)
                    WindConstant(
                        speed=us.UnitfulVelocity(value=10, units=us.UnitTypesVelocity.mps), 
                        direction=us.UnitfulDirection(
                            value=90, units=us.UnitTypesDirection.degrees                  
                        ),
                    )
                ],
                ForcingType.RAINFALL: [         # block rainfall starting 18h after event start, duration 6h, intensity 10 mm/hr 
                    RainfallSynthetic(
                        timeseries = TimeseriesFactory.from_args(             
                                shape_type=ShapeType.block,
                                duration=us.UnitfulTime(
                                    value=6, units=us.UnitTypesTime.hours
                                ),
                                peak_time=us.UnitfulTime(
                                    value=21, units=us.UnitTypesTime.hours
                                ),
                                peak_value=us.UnitfulIntensity(
                                    value=rainfall, units=us.UnitTypesIntensity.mm_hr
                                ),
                            )
                        
                    )
                ],
                ForcingType.DISCHARGE: [         # constant discharge in Cooper River of 5000 cubic feet per second
                    DischargeConstant(
                        river=RiverModel(
                            name="cooper",
                            description="Cooper River",
                            x_coordinate=595546.3,
                            y_coordinate=3675590.6,
                            mean_discharge=us.UnitfulDischarge(
                                value=5000, units=us.UnitTypesDischarge.cfs
                            ),
                        ),
                        discharge=us.UnitfulDischarge(
                            value=5000, units=us.UnitTypesDischarge.cfs 
                        ),
                    )
                ],
                ForcingType.WATERLEVEL: [       
                    WaterlevelSynthetic(
                        surge=SurgeModel(       # storm surge with gaussian shape, duration 10 hours, peak at 24h after event start_time, peak height 3m
                            timeseries=TimeseriesFactory.from_args(             
                                shape_type=ShapeType.gaussian,
                                duration=us.UnitfulTime(
                                    value=10, units=us.UnitTypesTime.hours
                                ),
                                peak_time=us.UnitfulTime(
                                    value=24, units=us.UnitTypesTime.hours
                                ),
                                peak_value=us.UnitfulLength(
                                    value=surge, units=us.UnitTypesLength.meters
                                ),
                            )
                        ),
                        tide=TideModel(     # tide with 1m amplitude and 12.4h duration
                            harmonic_amplitude=us.UnitfulLength(
                                value=1, units=us.UnitTypesLength.meters
                            ),
                            harmonic_period=us.UnitfulTime(
                                value=12.4, units=us.UnitTypesTime.hours
                            ),
                            harmonic_phase=us.UnitfulTime(
                                value=0, units=us.UnitTypesTime.hours
                            ),
                        ),
                    )
                ],
            },
        )
    )

print(events)

## **Step 3**: Compile and save event set
In this step, we create an event set from the subevents created in Step 2 and write the files to the input folder. This event set can then be used like any other event in a Scenario. Click [here](scenario.ipynb) to see examples on how to build a scenario. 

#TODO update link to scenario examples

In [20]:
from flood_adapt.objects.events.event_set import EventSet

subevents = []
for event, frequency in zip(events,freq):
    subevents.append(SubEventModel(name=event.name,frequency=frequency))

event_set = EventSet(
    name = name,
    sub_events=subevents    
)

print(event_set)

name='event_set' description='' mode=<Mode.risk: 'risk'> sub_events=[SubEventModel(name='subevent_1', frequency=0.1), SubEventModel(name='subevent_2', frequency=2.0), SubEventModel(name='subevent_3', frequency=0.05)]
